### In this notebook we prepare the data and create a new sudent dataset to find out student engagement 
#### Attributes: userID, major, industry interests, GPA, documents num
#### Potential Target Var: Job number, Events attendance, Appointments

In [65]:
# load all datasets
student_fname = 'data/Students.csv'
applications_fname = 'data/Applications.csv'
appointments_fname = 'data/Appointments.csv'
career_fair_fname = 'data/Career_Fair.csv'
#event_fname= 'Events.csv'

In [66]:
import pandas as pd
import numpy as np

In [67]:
df_student = pd.read_csv(student_fname)
df_applications = pd.read_csv(applications_fname)
df_appointments = pd.read_csv(appointments_fname)
df_fair = pd.read_csv(career_fair_fname)

In [68]:
def diff_dates(df, recent_date,old_date):
    df[recent_date] = pd.to_datetime(df[recent_date])
    df[old_date] = pd.to_datetime(df[old_date])
    new_col = df[recent_date] - df[old_date]
    new_col= new_col.dt.days
    
    return new_col

##### Cleaning appointments data

In [69]:
df_appointments.columns

Index(['Student Username', 'Staff Member ID',
       'Appointments Checked In? (Yes / No)',
       'Appointments Drop-in? (Yes / No)', 'Appointments Start Date Date',
       'Appointment Type Length (Minutes)', 'Appointment Type Name',
       'Appointment Categories Name', 'Created By Created At Date'],
      dtype='object')

In [70]:
df_appointments['Drop_in_advisor'] = np.where(df_appointments['Appointments Drop-in? (Yes / No)'] == 'No',0,1)

In [71]:
app_cols = ['Student Username','Drop_in_advisor','Appointment Type Length (Minutes)']
df_appointment1 = df_appointments[app_cols]

In [72]:
df_appointment2 = df_appointment1.groupby(by='Student Username').mean().reset_index()

#### Cleaning applications data

In [73]:
df_applications.shape

(51845, 7)

In [74]:
df_applications.columns

Index(['Applicant (student) Username', 'Applications ID',
       'Employer Industry Name', 'Postings Apply Start Date',
       'Postings Expiration Date Date', 'Student Educations Cumulative Gpa',
       'Applications Created At Date'],
      dtype='object')

In [75]:
df_applications['days_before_due']= diff_dates(df_applications,'Postings Expiration Date Date','Applications Created At Date')

#### Cleaning Career Fair Data

In [76]:
df_fair.shape

(4264, 10)

In [77]:
df_fair.columns

Index(['Student Attendees Username', 'Career Fair ID',
       'Career Fair Session Session Start Date',
       'Career Fair Student Registration Start Date',
       'Career Fair Student Registration End Date',
       'Career Fair Session Attendees Checked In? (Yes / No)',
       'Career Fair Session Attendees Checked In At Date',
       'Career Fair Session Attendees Created At Date',
       'Career Fair Session Attendees Pre-Registered At Date',
       'Career Fair Session Attendees Pre-Registered? (Yes / No)'],
      dtype='object')

In [78]:
df_fair['pre_reg']= np.where(df_fair['Career Fair Session Attendees Pre-Registered? (Yes / No)']=='Yes',1,0)
df_fair['check_in'] = np.where(df_fair['Career Fair Session Attendees Checked In? (Yes / No)']=='Yes',1,0)

In [79]:
fair_cols = ['Student Attendees Username','Career Fair ID','pre_reg','check_in']
df_fair1 = df_fair[fair_cols].groupby(by = 'Student Attendees Username').agg({'Career Fair ID':'count','pre_reg':'mean','check_in':'mean'}).reset_index()

##### Cleaning student data

In [80]:
df_student.shape

(21259, 8)

In [81]:
df_student.columns

Index(['Students Username', 'Students Gender', 'Work Authorization Name',
       'School Year Name', 'Career Interests: Career Clusters Name',
       'Majors Name', 'Educations Cumulative Gpa', 'Documents Count'],
      dtype='object')

In [82]:
## Making a new column that says whether student is at all engaged or not
df_student['engaged'] = 0

In [83]:
df_student['US Citizen']=np.where(df_student['Work Authorization Name']=='U.S. Citizen',1,0 )

In [84]:
student_cols = ['Students Username', 'Students Gender', 'US Citizen',
       'School Year Name', 'Career Interests: Career Clusters Name',
       'Majors Name', 'Educations Cumulative Gpa', 'Documents Count']

In [85]:
#compare with appointments
df_student1 = df_student[student_cols]

In [86]:
df_student1= df_student1.drop_duplicates(subset='Students Username', keep="last")

In [87]:
df_student1.columns

Index(['Students Username', 'Students Gender', 'US Citizen',
       'School Year Name', 'Career Interests: Career Clusters Name',
       'Majors Name', 'Educations Cumulative Gpa', 'Documents Count'],
      dtype='object')

In [88]:
df_appointment2.columns

Index(['Student Username', 'Drop_in_advisor',
       'Appointment Type Length (Minutes)'],
      dtype='object')

#### Joining all data

In [89]:
df_all_s_apt = pd.merge(df_student1, df_appointment2, how='left', left_on='Students Username',right_on= 'Student Username')

In [90]:
df_all_s_apt_app = pd.merge(df_all_s_apt, df_applications, how='left', left_on='Students Username',right_on= 'Applicant (student) Username')

In [91]:
df_all = pd.merge(df_all_s_apt_app, df_fair1, how='left', left_on='Students Username',right_on= 'Student Attendees Username')

In [92]:
df_all.columns

Index(['Students Username', 'Students Gender', 'US Citizen',
       'School Year Name', 'Career Interests: Career Clusters Name',
       'Majors Name', 'Educations Cumulative Gpa', 'Documents Count',
       'Student Username', 'Drop_in_advisor',
       'Appointment Type Length (Minutes)', 'Applicant (student) Username',
       'Applications ID', 'Employer Industry Name',
       'Postings Apply Start Date', 'Postings Expiration Date Date',
       'Student Educations Cumulative Gpa', 'Applications Created At Date',
       'days_before_due', 'Student Attendees Username', 'Career Fair ID',
       'pre_reg', 'check_in'],
      dtype='object')

In [93]:
df_all = df_all.drop(columns=['Student Educations Cumulative Gpa'])

In [94]:
df_all['Engaged_Fair'] = np.where(df_all['Student Attendees Username'].isnull(), 0,1)

In [95]:
df_all['Engaged_Appointment'] = np.where(df_all['Student Username'].isnull(), 0,1)

In [96]:
df_all['Engaged_Jobs'] = np.where(df_all['Applicant (student) Username'].isnull(), 0,1)

In [97]:
df_all = df_all.drop(columns=['Student Attendees Username','Student Username','Applicant (student) Username'])

In [98]:
df_all_1= df_all.drop_duplicates(subset='Students Username', keep="last")

In [99]:
df_all_1.head()

,Students Username,Students Gender,US Citizen,School Year Name,Career Interests: Career Clusters Name,Majors Name,Educations Cumulative Gpa,Documents Count,Drop_in_advisor,Appointment Type Length (Minutes),...,Postings Apply Start Date,Postings Expiration Date Date,Applications Created At Date,days_before_due,Career Fair ID,pre_reg,check_in,Engaged_Fair,Engaged_Appointment,Engaged_Jobs
104,amai2,Male,1,Alumni,"Engineering, Technology, Data & Physical Science",Business,3.77,99,0.000000,35.000000,...,2016-08-14,2016-12-31,2016-12-03,28.0,5.0,1.000000,0.40,1,1,1
291,dorenste,Male,1,Alumni,"Arts, Entertainment, Communications, Marketing...",Financial Economics,3.63,67,0.166667,28.714286,...,2016-07-31,2016-09-18,2016-09-12,6.0,2.0,0.500000,0.50,1,1,1
394,emill28,Male,1,Alumni,"Consulting, Finance, Operations, & Sales",Financial Economics,3.77,65,0.000000,30.000000,...,2016-09-01,2016-11-01,2016-10-22,10.0,4.0,1.000000,0.75,1,1,1
696,msingh20,Male,0,Alumni,"Engineering, Technology, Data & Physical Science",Economics,3.90,61,0.058824,29.117647,...,2016-08-17,2016-10-16,2016-09-09,37.0,4.0,0.000000,1.00,1,1,1
815,ssalen2,Female,1,Masters,"Consulting, Finance, Operations, & Sales",Data Science,NaN,61,0.103448,29.482759,...,2016-07-31,2016-10-16,2016-10-10,6.0,6.0,0.666667,1.00,1,1,1


In [102]:
df_all_1.

Index(['Students Username', 'Students Gender', 'US Citizen',
       'School Year Name', 'Career Interests: Career Clusters Name',
       'Majors Name', 'Educations Cumulative Gpa', 'Documents Count',
       'Drop_in_advisor', 'Appointment Type Length (Minutes)',
       'Applications ID', 'Employer Industry Name',
       'Postings Apply Start Date', 'Postings Expiration Date Date',
       'Applications Created At Date', 'days_before_due', 'Career Fair ID',
       'pre_reg', 'check_in', 'Engaged_Fair', 'Engaged_Appointment',
       'Engaged_Jobs'],
      dtype='object')

In [103]:
df_all_1.to_csv('Mega_Data.csv')

In [105]:
df_all_2=df_all_1.drop(columns = ['Career Interests: Career Clusters Name','Majors Name',
                         'Applications ID','Employer Industry Name',
                         'Postings Apply Start Date', 'Postings Expiration Date Date',
                         'Applications Created At Date'])

In [107]:
df_all_2['School Year Name'].unique()

array(['Alumni', 'Masters', 'Senior', 'Junior', 'Sophomore', 'Freshman',
       'Doctorate', nan, 'Masters of Business Administration',
       'Postdoctoral Studies'], dtype=object)

In [109]:
Year = {'Freshman':1, 'Sophomore':2, 'Junior':3, 'Senior':4,'Masters':5, 'Alumni':5,
        'Doctorate':5,'Masters of Business Administration':5,'Postdoctoral Studies':5}
df_all_2 = df_all_2.replace({'School Year Name': Year})

In [114]:
df_all_2 = df_all_2.drop(columns = ['Students Gender'])

In [115]:
df_all_2.to_csv('data/all_data_numeric.csv')